***GENERATED CODE FOR dataprep PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class DBFSConnector:

    def fetch(spark, config):
        df = spark.read.\
            options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                    inferschema='true',
                    delimiter=eval(config)["delimiter"])\
            .csv(eval(config)['url'])
        display(df.limit(2).toPandas())
        return df

    def put(df, path):
        df.to_csv("/dbfs" + eval(config)['url'], header=True, index=False)
        return True


***OPERATION FUNCTIONS***

In [ ]:
import pyspark
from dask.dataframe import from_pandas
import json


def replaceValues(df, functionsData, applyOn):
    for columnData in applyOn:
        for functionData in functionsData:
            if functionData['replaceType'] == 'value':
                if functionData['replaceValueType'] == 'min':
                    minValue = df[columnData['columnName']].min().compute()
                    df[columnData['columnName']] = df[columnData['columnName']].replace(
                        toReplace, minValue)
                elif functionData['replaceValueType'] == 'max':
                    maxValue = df[columnData['columnName']].max().compute()
                    df[columnData['columnName']] = df[columnData['columnName']].replace(
                        toReplace, maxValue)
                else:
                    df[columnData['columnName']] = df[columnData['columnName']].replace(
                        toReplace, functionData['ReplaceWith'])
            elif functionData['replaceType'] == 'none':
                if functionData['replaceValueType'] == 'min':
                    minValue = df[columnData['columnName']].min().compute()
                    df[columnData['columnName']
                       ] = df[columnData['columnName']].replace("", minValue)
                    df[columnData['columnName']
                       ] = df[columnData['columnName']].fillna(minValue)
                elif functionData['replaceValueType'] == 'max':
                    maxValue = df[columnData['columnName']].max().compute()
                    df[columnData['columnName']
                       ] = df[columnData['columnName']].replace("", maxValue)
                    df[columnData['columnName']
                       ] = df[columnData['columnName']].fillna(maxValue)
                else:
                    df[columnData['columnName']] = df[columnData['columnName']].replace(
                        "", functionData['ReplaceWith'])
                    df[columnData['columnName']] = df[columnData['columnName']].fillna(
                        functionData['ReplaceWith'])
    return df


def changeToString(df, functionsData, applyOn):
    for columnData in applyOn:
        df[columnData['columnName']] = df[columnData['columnName']].astype(str)
    return df


def runDataCleansing(sparkDf, spark, config):
    configObj = json.loads(config)
    sparkDf.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = from_pandas((sparkDf.toPandas()), npartitions=5)
    functionList = configObj['functionsApplied']
    Data_Cleansing_Methods = {"replaceBy": replaceValues,
                              "formula": calculateFormula,
                              "aggregate": aggregation,
                              "converttostringtype": changeToString,
                              "editname": renameColumns}
    for function in functionList:
        function['functionName']
        df = Data_Cleansing_Methods[function['functionName']](df, function['functionsData'],
                                                              function['applyOn'])
    sparkDf = spark.createDataFrame(df.compute())

    display(sparkDf.limit(2).toPandas())
    return sparkDf


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.functions import round, col
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoderEstimator
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def OHETransform(df, params, transformationData={}):
    feature = params["feature"]
    dfReturn = df
    dfReturn = dfReturn.fillna({feature: ''})
    outcolS = feature + "_toint"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcolS, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcolS).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcolS, dfReturn[outcolS].cast(IntegerType()))
        dfReturn = changed_type_df

    outcol = feature + "_onehotencoder"
    encoder = OneHotEncoderEstimator(inputCols=[outcolS], outputCols=[
                                     outcol], handleInvalid="keep")
    model = encoder.fit(dfReturn)
    dfReturn = model.transform(dfReturn)
    dfReturn = dfReturn.drop(outcolS)
    dfReturn = dfReturn.withColumn("final_col", to_array(dfReturn[outcol])) \
        .select(dfReturn.schema.names + [col("final_col")[0]])
    dfReturn = dfReturn.drop(outcol).withColumnRenamed("final_col[0]", outcol)
    dfReturn = dfReturn.withColumn(feature, round(dfReturn[outcol], 2))

    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = OHETransform(transformationDF, {'transformationsData': [{'feature_label': 'CHAS', 'transformation_label': 'One Hot Encoding'}], 'feature': 'CHAS', 'type': 'identifier', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                        'count': '43', 'mean': '0.07', 'stddev': '0.26', 'min': '0.0', 'max': '1.0', 'missing': '0'}, 'transformation': [{'transformation': 'One Hot Encoding', 'selectedAsDefault': 1}], 'updatedLabel': 'CHAS'}, {'feature_label': 'CHAS', 'transformation_label': 'One Hot Encoding'})
        transformationDF = transformationDF.drop('CHAS')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.model_selection import train_test_split
from tpot import TPOTRegressor
import pyspark


def functionRegression(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = sparkDF.toPandas()
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTRegressor(verbosity=3, generations=10, max_time_mins=5,
                              n_jobs=-1, random_state=25, population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Error rate of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

#%run dataprepHooks.ipynb
try:
	#sourcePreExecutionHook()

	dataprepdbfs = DBFSConnector.fetch(spark, "{'url': '/Demo/BostonTrain.csv', 'file_type': 'Delimeted', 'delimiter': ',', 'is_header': 'Use Header Line'}")
	#sourcePostExecutionHook(dataprepdbfs)

except Exception as ex: 
	logging.error(ex)


***PERFORMING OPERATIONS***

In [ ]:
#%run dataprepHooks.ipynb
try:
	#operationPreExecutionHook()

datapreparation = runDataCleansing(dataprepdbfs,spark,json.dumps( {"url": "/Demo/BostonTrain.csv", "source_attributes": null, "data_source": "dbfs", "BasicStats": {"missingValues": 0, "numberOfColumns": 14, "numberOfRows": 506, "duplicateRowCount": 0, "stats": [{"column": "CRIM", "alias": "CRIM", "type": "real", "max": "89.0", "min": "0.0", "mean": "8.6", "missing": 0, "stddev": 8.60154510533249, "checked": false}, {"column": "ZN", "alias": "ZN", "type": "real", "max": "100.0", "min": "0.0", "mean": "23.3", "missing": 0, "stddev": 23.32245299451514, "checked": false}, {"column": "INDUS", "alias": "INDUS", "type": "real", "max": "27.7", "min": "0.5", "mean": "6.9", "missing": 0, "stddev": 6.860352940897585, "checked": false}, {"column": "CHAS", "alias": "CHAS", "type": "real", "max": "1.0", "min": "0.0", "mean": "0.3", "missing": 0, "stddev": 0.25399404134041037, "checked": false}, {"column": "NOX", "alias": "NOX", "type": "real", "max": "0.9", "min": "0.4", "mean": "0.1", "missing": 0, "stddev": 0.11587767566755597, "checked": false}, {"column": "RM", "alias": "RM", "type": "real", "max": "8.8", "min": "3.6", "mean": "0.7", "missing": 0, "stddev": 0.7026171434153232, "checked": false}, {"column": "AGE", "alias": "AGE", "type": "real", "max": "100.0", "min": "2.9", "mean": "28.1", "missing": 0, "stddev": 28.14886140690362, "checked": false}, {"column": "DIS", "alias": "DIS", "type": "real", "max": "12.1", "min": "1.1", "mean": "2.1", "missing": 0, "stddev": 2.1057101266276104, "checked": false}, {"column": "RAD", "alias": "RAD", "type": "real", "max": "24.0", "min": "1.0", "mean": "8.7", "missing": 0, "stddev": 8.707259384239366, "checked": false}, {"column": "TAX", "alias": "TAX", "type": "real", "max": "711.0", "min": "187.0", "mean": "168.5", "missing": 0, "stddev": 168.53711605495906, "checked": false}, {"column": "PTRATIO", "alias": "PTRATIO", "type": "real", "max": "22.0", "min": "12.6", "mean": "2.2", "missing": 0, "stddev": 2.164945523714441, "checked": false}, {"column": "B", "alias": "B", "type": "real", "max": "396.9", "min": "0.3", "mean": "91.3", "missing": 0, "stddev": 91.29486438415783, "checked": false}, {"column": "LSTAT", "alias": "LSTAT", "type": "real", "max": "38.0", "min": "1.7", "mean": "7.1", "missing": 0, "stddev": 7.141061511348571, "checked": false}, {"column": "MEDV", "alias": "MEDV", "type": "real", "max": "50.0", "min": "5.0", "mean": "9.2", "missing": 0, "stddev": 9.197104087379818, "checked": false}]}, "startListenerOnly": 1, "FilePath": "/FileStore/platform/extra/5f91b8d5bd7fe75b99b821f91603386009/0part.csv", "HasBasicStats": 1, "functionsApplied": [{"functionName": "replaceBy", "applyOn": [{"columnName": "ZN", "type": "real", "min": "0.0", "max": "100.0", "mean": "23.3"}], "functionsData": [{"replaceType": "none", "toReplace": "", "replaceValueType": "min", "ReplaceWith": ""}]}, {"functionName": "converttostringtype", "applyOn": [{"columnName": "CHAS", "type": "real", "min": "0.0", "max": "0.0", "mean": "0.0"}], "functionsData": [{"dataType": "identifier"}]}], "fileheader": [{"field": "CRIM", "alias": "CRIM", "position": 1, "type": "real"}, {"field": "ZN", "alias": "ZN", "position": 2, "type": "real"}, {"field": "INDUS", "alias": "INDUS", "position": 3, "type": "real"}, {"field": "CHAS", "alias": "CHAS", "position": 4, "type": "real"}, {"field": "NOX", "alias": "NOX", "position": 5, "type": "real"}, {"field": "RM", "alias": "RM", "position": 6, "type": "real"}, {"field": "AGE", "alias": "AGE", "position": 7, "type": "real"}, {"field": "DIS", "alias": "DIS", "position": 8, "type": "real"}, {"field": "RAD", "alias": "RAD", "position": 9, "type": "real"}, {"field": "TAX", "alias": "TAX", "position": 10, "type": "real"}, {"field": "PTRATIO", "alias": "PTRATIO", "position": 11, "type": "real"}, {"field": "B", "alias": "B", "position": 12, "type": "real"}, {"field": "LSTAT", "alias": "LSTAT", "position": 13, "type": "real"}, {"field": "MEDV", "alias": "MEDV", "position": 14, "type": "real"}], "requestRatio": 1, "totalRows": 506, "functionChanges": [{"columnName": "ZN", "functionName": "Replace Outliers", "Type": "real", "Parameters": [{"replaceType": "none", "toReplace": "", "replaceValueType": "min", "ReplaceWith": ""}]}, {"columnName": "ZN", "functionName": "Replace Outliers", "Type": "real", "Parameters": [{"replaceType": "none", "toReplace": "", "replaceValueType": "min", "ReplaceWith": ""}]}, {"columnName": "CHAS", "functionName": "Change Datatype", "Type": "real", "Parameters": [{"dataType": "identifier"}]}]}))
	#operationPostExecutionHook(datapreparation)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run dataprepHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(datapreparation,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "CRIM", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "43", "mean": "1.4", "stddev": "3.4", "min": "0.01096", "max": "14.4383", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "CRIM"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "ZN", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "43", "mean": "19.86", "stddev": "31.04", "min": "0.0", "max": "100.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "ZN"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "INDUS", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "43", "mean": "9.19", "stddev": "6.01", "min": "0.46", "max": "21.89", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "INDUS"}, {"transformationsData": [{"feature_label": "CHAS", "transformation_label": "One Hot Encoding"}], "feature": "CHAS", "type": "identifier", "selected": "True", "replaceby": "max", "stats": {"count": "43", "mean": "0.07", "stddev": "0.26", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "One Hot Encoding", "selectedAsDefault": 1}], "updatedLabel": "CHAS"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "NOX", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "43", "mean": "0.51", "stddev": "0.09", "min": "0.389", "max": "0.74", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "NOX"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "RM", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "43", "mean": "6.33", "stddev": "0.55", "min": "5.344", "max": "8.247", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "RM"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "AGE", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "43", "mean": "60.83", "stddev": "28.55", "min": "6.5", "max": "100.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "AGE"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DIS", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "43", "mean": "4.63", "stddev": "2.39", "min": "1.4655", "max": "12.1265", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "DIS"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "RAD", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "43", "mean": "7.35", "stddev": "6.91", "min": "1.0", "max": "24.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "RAD"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "TAX", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "43", "mean": "366.49", "stddev": "134.79", "min": "187.0", "max": "666.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "TAX"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PTRATIO", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "43", "mean": "17.83", "stddev": "2.32", "min": "13.0", "max": "21.2", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PTRATIO"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "B", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "43", "mean": "349.27", "stddev": "109.4", "min": "2.52", "max": "396.9", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "B"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "LSTAT", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "43", "mean": "11.06", "stddev": "5.71", "min": "2.97", "max": "24.16", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "LSTAT"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "MEDV", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "43", "mean": "23.04", "stddev": "7.79", "min": "10.2", "max": "50.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "MEDV"}, {"feature": "CHAS_0_onehotencoder_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "43", "mean": "0.93", "stddev": "0.26", "min": "0.0", "max": "1.0", "missing": "0"}, "updatedLabel": "CHAS_0_onehotencoder_tran..."}, {"feature": "CHAS_1_onehotencoder_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "43", "mean": "0.07", "stddev": "0.26", "min": "0.0", "max": "1.0", "missing": "0"}, "updatedLabel": "CHAS_1_onehotencoder_tran..."}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run dataprepHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionRegression(autofe, ["CRIM", "ZN", "INDUS", "CHAS_onehotencoding", "NOX", "RM", "AGE", "DIS", "RAD", "TAX", "PTRATIO", "B", "LSTAT"], "MEDV")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML ['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    df = df[columnNames]
    R2 = np.round(sklearn.metrics.r2_score(y_test, y_predicted), 1)
    Mean_Squared_Error = np.round(sklearn.metrics.mean_squared_error(y_test, y_predicted), 1)
    Mean_Absolute_Error = np.round(sklearn.metrics.mean_absolute_error(y_test, y_predicted), 1)
    display(" R2 score of Prediction on test data    : %s"%R2)
    display(" Mean Squared Error of Prediction on test data    : %s"%Mean_Squared_Error)
    display(" Mean Absolute Error of Prediction on test data   : %s"%Mean_Absolute_Error)
    display(df.head())
except Exception as ex:
    logging.error(ex)

